In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv("spam.csv", encoding="latin-1")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
data['class']=data['class'].map({'ham':0, 'spam':1})

In [ ]:
data.head()

In [ ]:
#  !python3 -m pip install scikit-learn

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X=data['message']
y=data['class']

In [13]:
X.shape

(5572,)

In [14]:
y.shape

(5572,)

In [15]:
data.isnull().sum()

class      0
message    0
dtype: int64

In [16]:
cv=CountVectorizer()

In [17]:
X=cv.fit_transform(X)

In [18]:
x_train, x_test,y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [19]:
x_train.shape

(4457, 8672)

In [20]:
x_test.shape

(1115, 8672)

In [21]:
from sklearn.naive_bayes import MultinomialNB

In [22]:
model=MultinomialNB()

In [23]:
model.fit(x_train, y_train)

MultinomialNB()

In [24]:
model.score(x_test, y_test)

0.97847533632287

In [25]:
msg="Hi $"
data = [msg]
vect = cv.transform(data).toarray()
my_prediction = model.predict(vect)
print(my_prediction)

def result(msg):
    data = [msg]
    vect = cv.transform(data).toarray()
    my_prediction = model.predict(vect)
    if my_prediction[0]==1:
        #speak("This is a Spam mail")
        print("This is a Spam mail")
    else:
        #speak("This is not a Spam mail")
        print("This is not a Spam mail")
result(msg)

[1]
This is a Spam mail


In [26]:
vect

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [27]:
import pickle
pickle.dump(model, open('spam.pkl','wb'))
model1 = pickle.load(open('spam.pkl','rb'))


In [28]:
model1

MultinomialNB()

In [29]:
from win32com.client import Dispatch

In [30]:
def speak(text):
	speak=Dispatch(("SAPI.SpVoice"))
	speak.Speak(text)

In [31]:
def result(msg):
    data = [msg]
    vect = cv.transform(data).toarray()
    my_prediction = model1.predict(vect)
    if my_prediction[0]==1:
        speak("This is a Spam mail")
        print("This is a Spam mail")
    else:
        speak("This is not a Spam mail")
        print("This is not a Spam mail")

In [32]:
import tkinter as tk

In [33]:
root=tk.Tk()
root.geometry("200x200")
l2=tk.Label(root, text="Email Spam Classification Application")
l2.pack()
l1=tk.Label(root, text="Enter Your Message:")
l1.pack()
text=tk.Entry(root)
text.pack()
def result():
    data = [text.get()]
    vect = cv.transform(data).toarray()
    my_prediction = model1.predict(vect)
    if my_prediction[0]==1:
        speak("This is a Spam mail")
        print("This is a Spam mail")
    else:
        speak("This is not a Spam mail")
        print("This is not a Spam mail")
B=tk.Button(root, text="Click", command=result)
B.pack()

root.mainloop()

SOCKET

In [ ]:
#Server Code:

import socket
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load the spam dataset
data = pd.read_csv("spam.csv", encoding="latin-1")
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
data['class'] = data['class'].map({'ham':0, 'spam':1})

# Preprocess the data
cv = CountVectorizer()
X = cv.fit_transform(data['message'])
y = data['class']

# Train the model
model = MultinomialNB()
model.fit(X, y)

# Set up the socket
HOST = '127.0.0.1'  # localhost
PORT = 65432        # arbitrary non-privileged port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            # Receive the message from the client
            data = conn.recv(1024).decode()
            if not data:
                break

            # Transform the message into a vector and predict the class
            vect = np.array([int(x) for x in data.split(',')])
            prediction = model.predict(vect.reshape(1, -1))[0]

            # Send the prediction back to the client
            if prediction == 1:
                conn.sendall("This is a Spam mail".encode())
            else:
                conn.sendall("This is not a Spam mail".encode())

CLIENT


In [ ]:
import socket

# Set up the socket
HOST = '127.0.0.1'  # localhost
PORT = 5000        # arbitrary non-privileged port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    print(f"Connected to {HOST}:{PORT}...")

    # Send a message to the server
    msg = "Hi, you just won $400 $"
    s.sendall(msg.encode())

    # Receive the prediction from the server
    response = s.recv(1024).decode()
    print(f"Server response: {response}")